## Import requirements

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

import os
import glob
import pickle

import sys
sys.path.insert(0, 'convoca')
from ca_funcs import make_glider, make_game_of_life, make_ca, conv_cast
from utils import *
from convoca.train_ca import *


%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

# Build and train a model

In [ ]:
import numpy as np
import os
import glob
    
train_size, wspan, hspan = (100, 10, 10)
img_dir = "images/"


data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
data = []

PIL is a light weight image library and easy to install.  OpenCV is a beast and hard to install.  The opencv method does correct image conversion and leads to good training.  The PIL method does not yet. Suspect the issue is that opencv preserves the 3byte greyscale and and PIL converts grey to 1-byte.

In [ ]:
from PIL import Image

In [ ]:
for f1 in files:
    img = Image.open(f1).convert('L')
    img = img.resize((10,10))
    data.append(np.array(img))
   

In [ ]:
import cv2

In [ ]:
for f1 in files:
    img = cv2.imread(f1)
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img3 = cv2.resize(img2,(10,10))
    data.append(img3)
   

In [ ]:
     
X_train = np.array(data,np.float32)/255

X_train = tf.convert_to_tensor(X_train,tf.float32)

gol = make_game_of_life()

Y_train = gol(tf.convert_to_tensor(X_train, tf.float32))

X_train = X_train[..., tf.newaxis]
Y_train = Y_train[..., tf.newaxis]

In [ ]:
X_train.shape

In [ ]:
Y_train.shape

In [ ]:
#### Define and build model
layer_dims = [10, 10, 10]
num_classes = 2

try:
    del model
except:
    pass

loss = lambda x, y : tf.keras.losses.categorical_crossentropy(tf.reshape(x, shape=(-1, num_classes)), 
                                                              tf.reshape(y, shape=(-1, num_classes)), 
                                                              from_logits=True)
model = initialize_model((wspan, hspan), layer_dims, num_classes=num_classes)
# model = initialize_model((wspan, hspan), [10, 10, 10, 10], num_classes=num_classes, totalistic=True, bc="periodic")
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-2), loss=loss)

model.summary()

In [ ]:
#### Run training
Y_train_onehot = tf.squeeze(tf.one_hot(tf.cast(Y_train, tf.int32), num_classes))
train_history = model.fit(x=X_train, y=Y_train_onehot, epochs=50, batch_size=10, verbose=0,steps_per_epoch=5)

plt.plot(train_history.history['loss'], 'k')

In [ ]:
### Plot results

## Generate testing data
X_test = tf.convert_to_tensor(np.moveaxis(np.dstack([make_glider(10), make_glider(10)]), 2, 0), tf.float32)

Y_test = gol(X_test)
X_test = X_test[..., tf.newaxis]
Y_test = Y_test[..., tf.newaxis]

Y_pred = logit_to_pred(model(X_test), shape=(-1, wspan, hspan))

plt.figure(figsize=(12,4))

plt.subplot(1,3,1)
plt.imshow(tf.squeeze(X_test[0]))
plt.axis('off')
plt.title("Input")

plt.subplot(1,3,2)
plt.imshow(tf.squeeze(Y_test[0]))
plt.axis('off')
plt.title("Expected Output")

plt.subplot(1,3,3)
plt.imshow(tf.squeeze(Y_pred[0]))
plt.axis('off')
plt.title("Observed Output")


In [ ]:
type(X_test[0])

In [ ]:
### Save and load a model
model.save('path_to_my_model.h5')
#model = tf.keras.models.load_model('path_to_my_model.h5', custom_objects={'Wraparound2D': Wraparound2D})

# Show activation patterns of hidden layers

In [ ]:
import tensorflow.keras.backend as K

inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functor = K.function(inp, outputs)   # evaluation function

layer_outs = functor([X_test, 1.])



# Plot activations of different neurons in different layers 
all_layer_activations = list()

min_max_scaler = lambda x : (x - np.min(x))/(np.max(x) - np.min(x))
# min_max_scaler = lambda x : (x - np.mean(x))
for j in range(1, 5):
    if j==1:
        layer_im = np.hstack([min_max_scaler(layer_outs[1][0][..., i]) for i in range(10)])
    else:
        pattern = np.reshape(layer_outs[j][0], (wspan, hspan, -1))
        layer_im = np.hstack([min_max_scaler(pattern[..., i]) for i in range(10)])
    all_layer_activations.append(layer_im)

        
plt.figure()
plt.imshow(np.vstack(all_layer_activations))
plt.title("Activations of hidden layers given \"Glider\" input")

plt.figure()
plt.imshow(np.squeeze(np.dstack(model.layers[1].weights[0].numpy())))
plt.title("Convolutional filters")